In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context, getNode
from mand.core import ProfileMonitor, PrintMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import num, find
import datetime
from mand.lib.dbsetup import setUpDb

from mand.demos.trading import TradingBook, TradingPortfolio, MarketDataSource, MarketInterface

db = ObjectDb()
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x111b1bc50>: 137, mem=True, ro=False: entities=9, map=2


In [2]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['Eq-Prop', 'Eq-Inst', 'FX', 'Rates', 'Credit', 'Delta1', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 100
# children: 10


In [3]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[1]

b1 = p1.children()[0]
b2 = p2.children()[0]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop0
Eq-Inst0


In [4]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

In [5]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

In [6]:
with db:
    TradeOpenEvent = _tr.TradeOpenEvent
    cf1 = _tr.ForwardCashflow()
    ins1 = _tr.Equity()
    ins2 = _tr.Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

In [7]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            for f in footnotes:
                displayMarkdown('1. %s' % f)
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 1196.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [8]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

In [9]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 2216.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Book appears multiple times: /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|6,014,764|14|6,014,764|GetValue
|Report:data|1|6,014,750|444|6,014,750|GetValue/Calc
|TradingContainer:NPV|11|6,007,300|140|546,118|GetValue
|TradingContainer:NPV|11|6,007,159|1,818|546,105|GetValue/Calc
|Portfolio:items|121|5,817,749|1,375|48,080|GetValue
|Portfolio:items|121|5,817,666|19,321|48,079|GetValue/Calc
|Workbook:items|1,104|4,739,099|9,542|4,292|GetValue
|Workbook:items|1,100|4,729,557|4,584,030|4,299|GetValue/Calc
|Root:Clocks|2|1,643,125|2,601|821,562|Context
|Portfolio:children|242|1,052,678|1,397|4,349|GetValue
|Portfolio:children|121|1,051,280|528,497|8,688|GetValue/Calc
|Root:Start|1|501,363|18|501,363|Context
|Root:End|1|498,018|48|498,018|Context
|Root:Activity Portfolio|1|488,688|58|488,688|Context
|Root:Start breaks|1|487,349|40|487,349|Context
|Root:Activity Trading|1|483,737|46|483,737|Context
|Root:Amend Trading|1|480,942|40|480,942|Context
|Root:Activity MarketData|1|480,307|37|480,307|Context
|Root:Amend Portfolio|1|473,881|39|473,881|Context
|Root:Amend MarketData|1|472,851|39|472,851|Context
|PortfolioUpdateEvent:children|121|438,947|2,607|3,627|GetValue
|TradingBook|102|404,337|404,337|3,964|Db.Get
|Equity:NPV|15|187,420|164|12,494|GetValue
|Equity:NPV|15|187,255|435|12,483|GetValue/Calc
|MarketInterface:spot|15|174,128|129|11,608|GetValue
|MarketInterface:spot|15|173,998|414|11,599|GetValue/Calc
|ExternalRefData:state|15|164,396|146|10,959|GetValue
|ExternalRefData:state|15|164,249|295|10,949|GetValue/Calc
|RefData:state|15|163,954|287|10,930|GetValue
|RefData:state|15|163,666|71,311|10,911|GetValue/Calc
|Clock:cutoffs|2,480|111,144|7,488|44|GetValue
|Clock:cutoffs|20|103,720|429|5,186|GetValue/Calc
|Clock:parent|20|103,176|151|5,158|GetValue
|Clock:parent|20|103,024|83,078|5,151|GetValue/Calc
|PortfolioUpdateEvent|12|58,020|58,020|4,835|Db.Get
|RefDataUpdateEvent|9|43,515|43,515|4,835|Db.Get
|TradingPortfolio|10|40,395|40,395|4,039|Db.Get
|TradingBook:clock|2,200|37,398|18,328|16|GetValue
|Clock|5|20,774|20,774|4,154|Db.Get
|TradingBook:clock|1,100|19,069|15,197|17|GetValue/Calc
|TradeOpenEvent|4|18,644|18,644|4,661|Db.Get
|TradeOpenEvent:ticket|66|17,666|350|267|GetValue
|TradingTicket|4|17,315|17,315|4,328|Db.Get
|Equity:refdata|15|12,691|142|846|GetValue
|Equity:refdata|15|12,549|654|836|GetValue/Calc
|ClockEvent|2|10,210|10,210|5,105|Db.Get
|MarketInterface:source|15|9,187|108|612|GetValue
|MarketInterface:source|15|9,078|371|605|GetValue/Calc
|ClockEvent:parent|8|9,070|67|1,133|GetValue
|_WorkItemEvent:book2|66|8,685|293|131|GetValue
|MarketDataSource|2|8,540|8,540|4,270|Db.Get
|_WorkItemEvent:item|33|7,965|243|241|GetValue
|MarketInterface|2|7,768|7,768|3,884|Db.Get
|Equity|2|7,721|7,721|3,860|Db.Get
|Portfolio:clock|242|7,511|2,098|31|GetValue
|Portfolio:books|231|5,635|1,530|24|GetValue
|Portfolio:clock|121|5,413|1,629|44|GetValue/Calc
|MarketDataSource:clock|30|4,559|238|151|GetValue
|Portfolio:books|121|4,404|3,744|36|GetValue/Calc
|MarketDataSource:clock|15|4,321|206|288|GetValue/Calc
|TradeOpenEvent:premium|33|4,059|204|123|GetValue
|MarketInterface|1|4,034|4,034|4,034|Db.Put
|RootClock|1|4,025|4,025|4,025|Db.Get
|ForwardCashflow|1|3,854|3,854|3,854|Db.Get
|Event:amends|230|1,474|1,474|6|GetValue
|Entity:clock|40|483|301|12|GetValue
|_WorkItemEvent:book1|66|293|293|4|GetValue
|TradeOpenEvent:quantity|66|246|246|3|GetValue
|RefDataUpdateEvent:data|58|244|244|4|GetValue
|RootClock:cutoffs|52|186|186|3|GetValue
|Entity:clock|20|181|181|9|GetValue/Calc
|TradeOpenEvent:action|33|174|174|5|GetValue
|ForwardCashflow:NPV|11|171|127|15|GetValue
|MarketInterface:sourceName|15|166|118|11|GetValue
|TradeOpenEvent:unitPrice|33|156|156|4|GetValue
|Equity:assetName|15|91|91|6|GetValue
|MarketInterface:sourceName|15|47|47|3|GetValue/Calc
|ForwardCashflow:NPV|11|44|44|4|GetValue/Calc
|Report:valuable|1|3|3|3|GetValue
|Report:ts1|1|3|3|3|GetValue
|Report:ts2|1|2|2|2|GetValue